In [ ]:
import math
import sys

import IPython
import IPython.display as ipd
import matplotlib.pylab as plt
import numpy as np
import pandas as pd

%reload_ext autoreload
%autoreload 2

%matplotlib inline
#%matplotlib notebook

from matplotlib import rcParams
rcParams["figure.max_open_warning"] = False

In [ ]:
def custom_save(function=plt.savefig, fname=""):
    if not os.path.exists(os.path.dirname(fname)):
        os.makedirs(os.path.dirname(fname))
        print('created new directory', os.path.dirname(fname))
    try:
        # to make sure suptitle is not cut off
        function(fname, bbox_inches="tight")
    except:
        function(fname)
    print("saved as", fname)

def plot_spectrogram(spectrogram, label):
    plt.figure()
    plt.pcolormesh(np.log(spectrogram))
    plt.xlabel('time index [-]')
    plt.ylabel('frequency index [-]')
    plt.title(label)
    
def plot_soundlevel(df, label, ax=None):
    if ax is None:
        fig, ax = plt.subplots()
    ax.semilogy(df.timestamp_s, df.sound_level, label=label)
    ax.set_xlabel('time [s]')
    ax.set_ylabel('sound level')
    
def read_df_from_wav(fname):
    import wavio
    from scipy.signal import stft
    w = wavio.read(fname)
    source_data = w.data
    
    n_buffer = 1024

    f, t, source_stft = stft(source_data, w.rate, nperseg=n_buffer, axis=0)
    source_stft = source_stft.transpose([1, 0, 2]) # channels x frequencies x times
    source_freq = np.fft.rfftfreq(n=n_buffer, d=1/w.rate)

    df = pd.DataFrame(columns=df_source.columns)
    for i in range(source_stft.shape[2]):
        df.loc[len(df), :] = {
            "index": i,
            "timestamp": i * 1024 /w.rate * 1000, # miliseconds
            "n_mics": 2,
            "topic": "measurement_mic",
            "signals_f": source_stft[:, :, i], 
            "frequencies": source_freq,
            "n_frequencies": len(source_freq)
        }
    return df

In [ ]:
from evaluate_data import  CSV_DIRNAME, WAV_DIRNAME, FS 
from evaluate_data import read_df, get_spectrogram, add_soundlevel

## Compare sound levels

In [ ]:
duration = 10 * 1e3 # miliseconds from end
degree = 0
snr = False
props = False

df_source = read_df(degree=degree, props=False, snr=snr, motors=False, source=True)
df_all = read_df(degree=degree, props=props, snr=snr, motors=True, source=True)
df_props = read_df(degree=0, props=False, snr=False, motors=True, source=False)

spectrogram_source = get_spectrogram(df_source)
spectrogram_all = get_spectrogram(df_all)
spectrogram_props = get_spectrogram(df_props)

plot_spectrogram(spectrogram_source, label="source")
plot_spectrogram(spectrogram_all, label="all")
plot_spectrogram(spectrogram_props, label="props")

add_soundlevel(df_source, duration=duration)
add_soundlevel(df_all, duration=duration)
add_soundlevel(df_props, duration=duration)

fig, ax = plt.subplots()
plot_soundlevel(df_source, "source", ax=ax)
plot_soundlevel(df_all, "all", ax=ax)
plot_soundlevel(df_props, "props", ax=ax)
plt.legend()
plt.title('audio deck signals, selected 32 frequencies')

In [ ]:

fname_source = f'{WAV_DIRNAME}/crazyflie_audio_measurements_nomotors_nosnr_noprops_source_45.wav'
fname_all =    f'{WAV_DIRNAME}/crazyflie_audio_measurements_motors_nosnr_noprops_source_45.wav'
fname_props =  f'{WAV_DIRNAME}/crazyflie_audio_measurements_motors_nosnr_noprops_nosource_45.wav'

#df_wav_all = read_df_from_wav(fname_all) 
#df_wav_source = read_df_from_wav(fname_source) 
#df_wav_props = read_df_from_wav(fname_props) 
#df_wav_all.to_pickle(fname_all.replace('.wav', '.pk'))
#df_wav_source.to_pickle(fname_source.replace('.wav', '.pk'))
#df_wav_props.to_pickle(fname_props.replace('.wav', '.pk'))

df_wav_all = pd.read_pickle(fname_all.replace('.wav', '.pk'))
df_wav_source = pd.read_pickle(fname_source.replace('.wav', '.pk'))
df_wav_props = pd.read_pickle(fname_props.replace('.wav', '.pk'))

thresh = 1e2
add_soundlevel(df_wav_source, threshold=thresh)
add_soundlevel(df_wav_all, threshold=thresh)
add_soundlevel(df_wav_props, threshold=thresh)

fig, ax = plt.subplots()
plot_soundlevel(df_wav_source, label="source", ax=ax)
plot_soundlevel(df_wav_all, label="all", ax=ax)
plot_soundlevel(df_wav_props, label="props", ax=ax)
plt.legend()
plt.title('measurement mic signals, all 512 frequencies')

In [ ]:
def select_bins(row):
    signals_f = row.signals_f[:, bins_wav_selected]
    frequencies = row.frequencies[bins_wav_selected]
    row.frequencies = frequencies
    row.n_frequencies = len(frequencies)
    return row

freqs_selected = df_source.loc[0, 'frequencies']
freqs_wav = df_wav_source.loc[0, 'frequencies']
bins_wav_selected = [np.argmin(np.abs(freq - freqs_wav)) for freq in freqs_selected]

df_wav_selected_all = df_wav_all.apply(select_bins, axis=1)
df_wav_selected_source = df_wav_source.apply(select_bins, axis=1)
df_wav_selected_props = df_wav_props.apply(select_bins, axis=1)

print(f"reduced from {len(df_wav_all.loc[0, 'frequencies'])} to {len(df_wav_selected_all.loc[0, 'frequencies'])}")

thresh = 1e2
add_soundlevel(df_wav_selected_source, duration=duration, threshold=thresh)
add_soundlevel(df_wav_selected_all, duration=duration, threshold=thresh)
add_soundlevel(df_wav_selected_props, duration=duration, threshold=thresh)

fig, ax = plt.subplots()
plot_soundlevel(df_wav_selected_source, label="source", ax=ax)
plot_soundlevel(df_wav_selected_all, label="all", ax=ax)
plot_soundlevel(df_wav_selected_props, label="props", ax=ax)
plt.legend()
plt.title('measurement mic signals, selected 32 frequencies')

## Plot results

In [ ]:
import itertools
from evaluate_data import degree_list, method_list, combine_list, normalize_list

def filter_df_by_dict(result_df, this_dict, verbose=False):
    this_df = result_df.copy()
    for key, val in this_dict.items():
        if not len(this_df.loc[this_df[key]==val]):
            if verbose:
                print(f'did not find any {key}=={val} in results. Available: {this_df[key].unique()}')
        this_df = this_df.loc[this_df[key]==val]
        if not len(this_df): # can stop filtering, already empty
            break
    return this_df

def plot_spectrum(full_spectrum, title='', ax=None):
    angles = np.linspace(0, 360, full_spectrum.shape[0])
    if ax is None:
        fig, ax = plt.subplots()
    else:
        fig = plt.gcf()
        
    fig.set_size_inches(10, 5)
    ax.pcolormesh(range(full_spectrum.shape[1]), angles, np.log(full_spectrum))
    ax.set_xlabel('time idx [-]')
    ax.set_ylabel('angle [deg]')
    ax.set_title(title)

#fname = 'results/static_spectra.pkl'
fname = 'results/static_spectra_raw.pkl'
#fname = 'results/snr_test.pkl'
try:
    result_df = pd.read_pickle(fname)
except FileNotFoundError: 
    print('Did not find {fname}. Did you run evaluate_data.py? ')

## A. Ideal case: what is best combination of method+combine+normalize? 

In [ ]:
base_dict = dict(
    motors=False,
    props=False,
    snr=False, 
    degree=None, # will be set later
    method=None,
    combine=None,
    normalize=None
)

for degree, method, normalize, combine in itertools.product(degree_list, method_list, normalize_list, combine_list):
    gt_degree = 180 - degree
    
    this_dict = base_dict.copy()
    this_dict["degree"] = degree
    this_dict["method"] = method
    this_dict["combine"] = combine
    this_dict["normalize"] = normalize
    this_df = filter_df_by_dict(result_df, this_dict)
            
    if not len(this_df):
        continue 
        
    # angles x times
    full_spectrum = np.array([*this_df.loc[:, "spectrum"]]).T
    title = f"{degree} deg, method:{method}, normalize:{normalize}, combine:{combine}"
    
    fig, ax = plt.subplots()
    plot_spectrum(full_spectrum, title=title, ax=ax)
    ax.axhline(gt_degree, color='orange', ls=':')

## B. no motors: does it help to use high-SNR bins? 

In [ ]:
base_dict = dict(
    motors=False,
    props=False,
    # fill from above 
    method='mvdr',
    combine='sum',
    normalize='zero_to_one',
    snr=None, 
    degree=None, # will be set later
)

for degree, snr in itertools.product(degree_list, [False, True]):
    gt_degree = 180 - degree
    
    this_dict = base_dict.copy()
    this_dict["degree"] = degree
    this_dict["snr"] = snr 
    this_df = filter_df_by_dict(result_df, this_dict)
            
    if not len(this_df):
        continue 
        
    # angles x times
    full_spectrum = np.array([*this_df.loc[:, "spectrum"]]).T
    title = f"{degree} deg, snr:{snr}"
    
    fig, ax = plt.subplots()
    plot_spectrum(full_spectrum, title=title, ax=ax)
    ax.axhline(gt_degree, color='orange', ls=':')

## C. motors: what filtering helps? 

In [ ]:
import matplotlib.gridspec as gridspec
from audio_stack.spectrum_estimator import combine_rows
    
base_dict = dict(
    motors=True,
    method='mvdr',
    combine='sum',
    normalize='zero_to_one',
    degree=None, 
    snr=None, 
    props=None, # will be filled later
)

for degree, snr, props in itertools.product(degree_list, [False, True], [False, True]):
    gt_degree = 180 - degree
    this_dict = base_dict.copy()
    this_dict["degree"] = degree
    this_dict["snr"] = snr 
    this_dict["props"] = props
    this_df = filter_df_by_dict(result_df, this_dict)
            
    if not len(this_df):
        continue 
        
    # angles x times
    full_spectrum = np.array([*this_df.loc[:, "spectrum"]]).T
    title = f"{degree} deg, snr:{snr}, props:{props}"
    angles = np.linspace(0, 360, full_spectrum.shape[0])
    
    fig = plt.figure(constrained_layout=True)
    spec = gridspec.GridSpec(ncols=3, nrows=1, figure=fig, width_ratios=[3, 1, 1])
    ax0 = fig.add_subplot(spec[0, 0])
    ax1 = fig.add_subplot(spec[0, 1], sharey=ax0)
    ax2 = fig.add_subplot(spec[0, 2], sharey=ax0)
    
    plot_spectrum(full_spectrum, title=title, ax=ax0)
    ax0.axhline(gt_degree, color='orange', ls=':')
    
    ax1.plot(combine_rows(full_spectrum.T, "sum"), angles)
    ax1.axhline(gt_degree, color='orange', ls=':')
    ax1.set_title('sum')
    
    ax2.plot(combine_rows(full_spectrum.T, "product"), angles)
    ax2.axhline(gt_degree, color='orange', ls=':')
    ax2.set_title('product')
    #axs[1].invert_xaxis()

## D. Understand why SNR selection is not helping

In [ ]:
base_dict = dict(
    motors=True,
    method='mvdr',
    combine='sum',
    normalize='zero_to_one',
    degree=20, 
    props=True, 
    snr=None, # will be filled later
)

n_times = 4

for snr, in itertools.product([False, True]):
    this_dict = base_dict.copy()
    this_dict["snr"] = snr 
    this_df = filter_df_by_dict(result_df, this_dict, verbose=False)
    
    if not len(this_df):
        continue 
        
    raw_spectrum = np.array([*this_df.loc[:, "spectrum_raw"]]).T
    raw_spectrum = raw_spectrum.transpose(0, 2, 1) # angles x times x frequencies
    
    gt_degree = 180 - this_dict["degree"]
    angles = np.linspace(0, 360, raw_spectrum.shape[0])
    
    fig, axs = plt.subplots(1, n_times+2, sharey=True)
    fig.suptitle(f"snr:{snr}")
    fig.set_size_inches(10, 5)
    for i in range(n_times):
        raw_spectrum_flat = raw_spectrum[:, -i, :]
        
        frequencies = this_df.iloc[i].frequencies
        axs[i].pcolormesh(frequencies, angles, raw_spectrum_flat)
        axs[i].set_title(f"time -{i}")
        axs[i].axhline(gt_degree, color='orange', ls=':')
        
    raw_spectrum_sum = np.sum(raw_spectrum[:, -n_times:, :].reshape(raw_spectrum.shape[0],-1), axis=1)
    raw_spectrum_product = np.sum(np.log10(raw_spectrum[:, -n_times:, :]).reshape(raw_spectrum.shape[0],-1), axis=1)
    axs[-2].plot(raw_spectrum_sum, angles)
    axs[-1].plot(raw_spectrum_product, angles)

## E. Understand which frequency bins the SNR scheme picks (and try to pick better ones)

In [ ]:
# should pick propeller signals:
df_snr_noprops = read_df(degree=0, props=False, snr=True, motors=True, source=True)

# should pick between propellers (but maybe still too close?)
df_snr_props = read_df(degree=0, props=True, snr=True, motors=True, source=True)

# should pick uniformly
df_nosnr_noprops = read_df(degree=0, props=False, snr=False, motors=True, source=True)

fname_props =  f'{WAV_DIRNAME}/crazyflie_audio_measurements_motors_nosnr_noprops_nosource_45.wav'
df_props = read_df_from_wav(fname_props)

In [ ]:
time = 0
fig, ax = plt.subplots()
for df, label in zip([df_snr_noprops, df_snr_props, df_nosnr_noprops], ["props", "between", "uniform"]):
    normalized_spec = np.sum(np.abs(df.loc[time].signals_f), axis=0)
    normalized_spec -= np.min(normalized_spec)
    normalized_spec /= np.max(normalized_spec)
    ax.scatter(df.loc[time].frequencies, normalized_spec, label=label)
    
normalized_spec = np.sum(np.abs(df_props.iloc[0].signals_f), axis=0)
normalized_spec -= np.min(normalized_spec)
normalized_spec /= np.max(normalized_spec)
ax.plot(df_props.iloc[0].frequencies, normalized_spec, color="C3", label='measurement mics')
#ax.set_yscale('log')
ax.legend()
ax.set_xlim([0, 4000])
fig.set_size_inches(10, 5)

## F. Average over time

Take the average of a few samples over time, considering the drone's movement. Plot the angle estimate vs. ground truth over time. 

In [ ]:
from audio_stack.beam_former import BeamFormer
from audio_stack.spectrum_estimator import normalize_rows

base_dict = dict(
    motors=True,
    method='mvdr',
    combine='sum',
    normalize='zero_to_one',
    degree=0, # 20 does not exist
    props=False, 
    snr=False, # will be filled later
)

n_times = 4
combination_n = 3
combination_method = "sum"
normalization_method = "none"

this_df = filter_df_by_dict(result_df, base_dict, verbose=True)

beam_former = BeamFormer()
beam_former.init_dynamic_estimate(combination_n, combination_method, normalization_method)

combined_heatmap = None
raw_heatmap = None

for i, row in this_df.iterrows():
    raw_spectrum = row.spectrum_raw # frequencies x angles
    
    # TODO(FD): read orientation from current pose estimate.
    beam_former.add_to_dynamic_estimates(raw_spectrum, orientation=0)
    estimate = beam_former.get_dynamic_estimate()
    
    combined_estimate = combine_rows(estimate, combination_method, keepdims=True)
    combined_estimate = normalize_rows(combined_estimate, method="zero_to_one")
    
    # TODO(FD): could also read this directly from df. Refactor? 
    raw_estimate = combine_rows(raw_spectrum, combination_method, keepdims=True)
    raw_estimate = normalize_rows(raw_estimate, method="zero_to_one")
    
    if combined_heatmap is None:
        combined_heatmap = combined_estimate.flatten()
    else:
        combined_heatmap = np.c_[combined_heatmap, combined_estimate.flatten()]
        
    if raw_heatmap is None:
        raw_heatmap = raw_estimate.flatten()
    else:
        raw_heatmap = np.c_[raw_heatmap, raw_estimate.flatten()]
    
fig, ax = plt.subplots()
ax.pcolormesh(range(combined_heatmap.shape[1]), angles, combined_heatmap)
ax.set_title('combined heatmap')

fig, ax = plt.subplots()
ax.pcolormesh(range(raw_heatmap.shape[1]), angles, raw_heatmap)
ax.set_title('raw heatmap')

In [ ]:
## plot angle estimate over time

In [ ]:
## try to combine different angles

In [ ]:
## plot which frequency bins yield the correct angle estimate